In [1]:
from pathlib import Path
import tensorflow as tf

SEED = 42
IMG_SIZE = (28, 28)
BATCH = 64

root = Path("data")
train_root = root / "train"
test_root  = root / "test"

# train / val (train에서 20%를 검증으로)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root,
    validation_split=0.2, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int"
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root,
    validation_split=0.2, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int"
)

# test는 train의 class_names 순서를 그대로 사용해서 라벨 정렬 문제 예방
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_root,
    image_size=IMG_SIZE, batch_size=BATCH, shuffle=False, label_mode="int",
    class_names=train_ds.class_names
)

print("classes:", train_ds.class_names)
num_classes = len(train_ds.class_names)

Found 300 files belonging to 3 classes.
Using 240 files for training.
Found 300 files belonging to 3 classes.
Using 60 files for validation.
Found 60 files belonging to 3 classes.
classes: ['paper', 'rock', 'scissor']


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

keras.backend.clear_session()

inputs = keras.Input(shape=IMG_SIZE + (3,))  # RGB
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomRotation(0.08)(x)
x = layers.RandomZoom(0.10)(x)
x = layers.RandomContrast(0.10)(x)
x = layers.Rescaling(1./255)(x)

# Block 1
x = layers.Conv2D(48, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(48, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.25)(x)

# Block 2
x = layers.Conv2D(96, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(96, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.30)(x)

# Block 3
x = layers.Conv2D(160, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(96, activation="relu")(x)
x = layers.Dropout(0.35)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 random_flip (RandomFlip)    (None, 28, 28, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 28, 28, 3)         0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 28, 28, 3)         0         
                                                                 
 random_contrast (RandomCon  (None, 28, 28, 3)         0         
 trast)                                                          
                                                                 
 rescaling (Rescaling)       (None, 28, 28, 3)         0     

In [3]:
ckpt = keras.callbacks.ModelCheckpoint(filepath="best.weights.h5", monitor="val_accuracy", mode="max",
                                       save_best_only=True, save_weights_only=True)

plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[ckpt, plateau], verbose=1)

Epoch 1/50
4/4 [==============================] - 3s 330ms/step - loss: 0.8660 - accuracy: 0.6000 - val_loss: 1.0838 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/50
4/4 [==============================] - 1s 241ms/step - loss: 0.5430 - accuracy: 0.8000 - val_loss: 1.0980 - val_accuracy: 0.3667 - lr: 0.0010
Epoch 3/50
4/4 [==============================] - 1s 222ms/step - loss: 0.3499 - accuracy: 0.8625 - val_loss: 1.1678 - val_accuracy: 0.3667 - lr: 0.0010
Epoch 4/50
4/4 [==============================] - ETA: 0s - loss: 0.1945 - accuracy: 0.9750
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 1s 221ms/step - loss: 0.1945 - accuracy: 0.9750 - val_loss: 1.4389 - val_accuracy: 0.3667 - lr: 0.0010
Epoch 5/50
4/4 [==============================] - 1s 254ms/step - loss: 0.1268 - accuracy: 0.9625 - val_loss: 1.8156 - val_accuracy: 0.3667 - lr: 5.0000e-04
Epoch 6/50
4/4 [==============================] - 1s 232ms/step - loss: 0.

In [4]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

test_opt = test_ds.cache()

# 예측
y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)

present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

# Confusion matrix
print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

# Classification report
print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present, target_names=target_names, digits=4))

# loss / accuracy
test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[ 0 20  0]
 [ 0 20  0]
 [ 0 20  0]]

Classification report:
              precision    recall  f1-score   support

       paper     0.0000    0.0000    0.0000        20
        rock     0.3333    1.0000    0.5000        20
     scissor     0.0000    0.0000    0.0000        20

    accuracy                         0.3333        60
   macro avg     0.1111    0.3333    0.1667        60
weighted avg     0.1111    0.3333    0.1667        60


[Test] loss=8.2081  acc=0.3333


c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

---

### 2nd 도전

In [5]:
# 소량 샘플로 과적합 테스트
small_train = train_ds.unbatch().take(60).batch(32)   
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
hist_small = model.fit(small_train, epochs=30, verbose=0)

print("small-set max acc:", max(hist_small.history["accuracy"]))

small-set max acc: 1.0


In [6]:
from collections import Counter

# 학습률 낮추기
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 클래스 가중치
from collections import Counter
cnt = Counter()
for _, yb in train_ds.unbatch():
    cnt[int(yb.numpy())] += 1
K = len(train_ds.class_names); total = sum(cnt.values())
class_weight = {i: total/(K*cnt.get(i,1)) for i in range(K)}
print("class_weight:", class_weight)

# 체크포인트(가중치만 저장) + Plateau
ckpt = keras.callbacks.ModelCheckpoint("best.weights.h5", monitor="val_accuracy",
                                       mode="max", save_best_only=True, save_weights_only=True)
plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=50,
                    callbacks=[ckpt, plateau], class_weight=class_weight, verbose=1)

model.load_weights("best.weights.h5")

class_weight: {0: 1.0, 1: 1.0256410256410255, 2: 0.975609756097561}
Epoch 1/50
4/4 [==============================] - 3s 336ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 14.5375 - val_accuracy: 0.3667 - lr: 3.0000e-04
Epoch 2/50
4/4 [==============================] - 1s 237ms/step - loss: 0.0149 - accuracy: 0.9958 - val_loss: 16.0678 - val_accuracy: 0.3667 - lr: 3.0000e-04
Epoch 3/50
4/4 [==============================] - 1s 247ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 18.3081 - val_accuracy: 0.3667 - lr: 3.0000e-04
Epoch 4/50
4/4 [==============================] - ETA: 0s - loss: 0.0018 - accuracy: 1.0000
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
4/4 [==============================] - 1s 253ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 18.4968 - val_accuracy: 0.3667 - lr: 3.0000e-04
Epoch 5/50
4/4 [==============================] - 1s 253ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 18.0226 - val_accuracy: 0.3667 - 

In [7]:
test_opt = test_ds.cache()

y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)
present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present, target_names=target_names, digits=4))

test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[ 0 20  0]
 [ 0 20  0]
 [ 0  6 14]]

Classification report:
              precision    recall  f1-score   support

       paper     0.0000    0.0000    0.0000        20
        rock     0.4348    1.0000    0.6061        20
     scissor     1.0000    0.7000    0.8235        20

    accuracy                         0.5667        60
   macro avg     0.4783    0.5667    0.4765        60
weighted avg     0.4783    0.5667    0.4765        60


[Test] loss=3.5010  acc=0.5667


c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

---

### 3rd 도전

In [8]:
# 증강 정의 
aug = keras.Sequential([layers.RandomFlip("horizontal"),
                        layers.RandomRotation(0.05),
                        layers.RandomZoom(0.08),
                        layers.RandomContrast(0.05)])

# 기존 모델을 백본으로 래핑
backbone = model                       
inputs = keras.Input(shape=(28, 28, 3))
x = aug(inputs)
x = layers.Rescaling(1./255)(x)
outputs = backbone(x)                  
model = keras.Model(inputs, outputs)   

# 재컴파일 
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ckpt = keras.callbacks.ModelCheckpoint("best.weights.h5", monitor="val_accuracy",
                                       mode="max", save_best_only=True, save_weights_only=True)
plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[ckpt, plateau],
                    class_weight=class_weight, verbose=1)

model.load_weights("best.weights.h5")

Epoch 1/50
4/4 [==============================] - 4s 354ms/step - loss: 0.2596 - accuracy: 0.9083 - val_loss: 4.3357 - val_accuracy: 0.3333 - lr: 3.0000e-04
Epoch 2/50
4/4 [==============================] - 1s 284ms/step - loss: 0.1604 - accuracy: 0.9500 - val_loss: 2.7060 - val_accuracy: 0.3333 - lr: 3.0000e-04
Epoch 3/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0675 - accuracy: 0.9625 - val_loss: 2.4504 - val_accuracy: 0.3000 - lr: 3.0000e-04
Epoch 4/50
4/4 [==============================] - 1s 297ms/step - loss: 0.0595 - accuracy: 0.9833 - val_loss: 2.5891 - val_accuracy: 0.3000 - lr: 3.0000e-04
Epoch 5/50
4/4 [==============================] - 1s 247ms/step - loss: 0.0378 - accuracy: 0.9833 - val_loss: 2.6775 - val_accuracy: 0.3000 - lr: 3.0000e-04
Epoch 6/50
4/4 [==============================] - ETA: 0s - loss: 0.0660 - accuracy: 0.9792
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
4/4 [==============================] - 1s 275ms

In [9]:
test_opt = test_ds.cache()

y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)
present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present, target_names=target_names, digits=4))

test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[20  0  0]
 [20  0  0]
 [20  0  0]]

Classification report:
              precision    recall  f1-score   support

       paper     0.3333    1.0000    0.5000        20
        rock     0.0000    0.0000    0.0000        20
     scissor     0.0000    0.0000    0.0000        20

    accuracy                         0.3333        60
   macro avg     0.1111    0.3333    0.1667        60
weighted avg     0.1111    0.3333    0.1667        60


[Test] loss=4.1049  acc=0.3333


c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

---

### 4th 도전

In [10]:
IMG_SIZE = (160, 160)  

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_root, image_size=IMG_SIZE, batch_size=BATCH,
    shuffle=False, label_mode="int", class_names=train_ds.class_names)

print("classes:", train_ds.class_names)

Found 300 files belonging to 3 classes.
Using 240 files for training.
Found 300 files belonging to 3 classes.
Using 60 files for validation.
Found 60 files belonging to 3 classes.
classes: ['paper', 'rock', 'scissor']


In [11]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

keras.backend.clear_session()

num_classes = len(train_ds.class_names)

augment = keras.Sequential([layers.RandomFlip("horizontal"),
                            layers.RandomRotation(0.05),
                            layers.RandomZoom(0.08),
                            layers.RandomContrast(0.05)], name="augment")

base = EfficientNetB0(include_top=False, weights="imagenet", input_shape=IMG_SIZE + (3,))
base.trainable = False   

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = augment(inputs)
x = preprocess_input(x)             
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ckpt = keras.callbacks.ModelCheckpoint("best.weights.h5", monitor="val_accuracy",
                                       mode="max", save_best_only=True, save_weights_only=True)
plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=8,
                    callbacks=[ckpt, plateau], verbose=1)

Epoch 1/8
4/4 [==============================] - 14s 2s/step - loss: 1.0498 - accuracy: 0.4500 - val_loss: 0.8205 - val_accuracy: 0.9333 - lr: 0.0010
Epoch 2/8
4/4 [==============================] - 6s 2s/step - loss: 0.7890 - accuracy: 0.7583 - val_loss: 0.6126 - val_accuracy: 0.9833 - lr: 0.0010
Epoch 3/8
4/4 [==============================] - 6s 2s/step - loss: 0.5922 - accuracy: 0.9500 - val_loss: 0.4520 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/8
4/4 [==============================] - 5s 1s/step - loss: 0.4455 - accuracy: 0.9792 - val_loss: 0.3436 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/8
4/4 [==============================] - 6s 1s/step - loss: 0.3604 - accuracy: 0.9750 - val_loss: 0.2732 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/8
4/4 [==============================] - 5s 1s/step - loss: 0.2735 - accuracy: 0.9958 - val_loss: 0.2234 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/8
4/4 [==============================] - 5s 1s/step - loss: 0.2184 - accuracy: 0.9917 - val_loss:

In [12]:
test_opt = test_ds.cache()
y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)
present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present,
                            target_names=target_names, digits=4))

test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[19  0  1]
 [ 0 20  0]
 [ 0  0 20]]

Classification report:
              precision    recall  f1-score   support

       paper     1.0000    0.9500    0.9744        20
        rock     1.0000    1.0000    1.0000        20
     scissor     0.9524    1.0000    0.9756        20

    accuracy                         0.9833        60
   macro avg     0.9841    0.9833    0.9833        60
weighted avg     0.9841    0.9833    0.9833        60


[Test] loss=0.3853  acc=0.9833
